## Libraries

In [1]:
import pandas as pd
import numpy as np

from tqdm.notebook import tqdm
import datatable as dt
import datetime
import string
import random
import glob
import time
import os
import gc

from scipy.stats import rankdata

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder

import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.utilities.model_summary import ModelSummary
from pytorch_lightning.loggers import TensorBoardLogger

import torch
from torch import nn
from torch.nn import functional as F
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import TensorDataset, DataLoader

import torchmetrics

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter('ignore')

## Parameter

In [2]:
class CFG:
    input = "../input/spaceship-titanic"
    target = 'Transported'

    n_splits = 10  # クロスバリデーションの分割数
    seed = 42

    batch_size = 1024  # 1回の学習で何レコードやるか
    workers = 4  # 並列処理数  大きいとエラー?
    epochs = 2000
    learning_rate = 1e-2

    factor = 0.8  # 学習率の逓減率
    min_lr = 1e-8

    lr_patience = 10  # 何ステップ良くならなければ、学習率を引き下げるか
    es_patience = 50

    pred = 'pred'
    test_pred = [f'pred_{i}' for i in range(n_splits)]  # 各foldの実行結果

    model_path = "models"
    tb_log_name = "lightning_logs"

## Data Loading

In [3]:
%%time

train = pd.read_csv("/".join([CFG.input, "train.csv"]))
test = pd.read_csv("/".join([CFG.input, "test.csv"]))
sub = pd.read_csv("/".join([CFG.input, "sample_submission.csv"]))

FileNotFoundError: [Errno 2] No such file or directory: '../input/spaceship-titanic/train.csv'

In [ ]:
display(train.shape)
display(test.shape)

(8693, 14)

(4277, 13)

In [ ]:
display(train.head())
display(test.head())

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


PassengerIdに意味はない

Nameに意味はない

欠落しているデータがある

Cabinは分けるべき

In [ ]:
train.nunique()

PassengerId     8693
HomePlanet         3
CryoSleep          2
Cabin           6560
Destination        3
Age               80
VIP                2
RoomService     1273
FoodCourt       1507
ShoppingMall    1115
Spa             1327
VRDeck          1306
Name            8473
Transported        2
dtype: int64

In [ ]:
train.isna().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [ ]:
train.dtypes

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

In [ ]:
all_df = pd.concat([train, test]).reset_index(drop=True)

In [ ]:
bool_features = ['CryoSleep', 'VIP', 'Transported']
for feature in bool_features:
    all_df[feature] = all_df[feature].astype(float)
    all_df[feature] = all_df[feature].fillna(-1)
    all_df[feature] = all_df[feature].astype(int)

In [ ]:
all_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,0,B/0/P,TRAPPIST-1e,39.0,0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0
1,0002_01,Earth,0,F/0/S,TRAPPIST-1e,24.0,0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1
2,0003_01,Europa,0,A/0/S,TRAPPIST-1e,58.0,1,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0
3,0003_02,Europa,0,A/0/S,TRAPPIST-1e,33.0,0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0
4,0004_01,Earth,0,F/1/S,TRAPPIST-1e,16.0,0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1


In [ ]:
class FE:
    # passengerId -> 落とす
    # HomePlanet -> fit_transform
    # CryoSleep -> fit_transform
    # cabin -> 文字で分ける
    # Age -> floatとして扱う
    # Vip -> fit_transform
    def __init__(self, df):
        self.df = df
        self.cabin_len = len(self.df['Cabin'][0])

    def convert_True_False(self, txt):
        if txt == True:
            return 1
        elif txt == False:
            return - 0
        else:
            return -1
        
    def fit_target(self):
        self.df[CFG.target] = self.df[CFG.target].apply(self.convert_True_False).astype(int)
        
    def c2i(self, c):
        return ord(c)

    def label_encoding(self, features):
        # 文字列の変数をfit_transformする
        new_features = []
        for feature in features:
            if self.df[feature].dtype == 'O':
                le = LabelEncoder()
                new_feature = f'{feature}_enc'
                self.df[new_feature] = le.fit_transform(self.df[feature])
                new_features.append(new_feature)
            else:
                new_features.append(feature)
        return new_features
    
    def sep_sharp(self, txt):
        arr = txt.split('/')
        return tuple(arr)

    def fill_all_na(self, features):
        for feature in features:
            if self.df[feature].dtype == 'O':
                self.df[feature] = self.df[feature].fillna('XXX')
            elif self.df[feature].dtype == 'bool':
                self.df[feature] = self.df[feature].fillna(False)
            else:
                self.df[feature] = self.df[feature].fillna(-1)
    
    def separete_cabin(self):
        # cabinを文字に分割して新しい列にする
        # 編集後は整数を入れる
        self.df['Cabin'] = self.df['Cabin'].fillna('X/X/X')
        self.df[['Cabin1', 'Cabin2', 'Cabin3']] = self.df.apply(lambda x: self.sep_sharp(x['Cabin']), axis=1, result_type='expand')

    def standard_scaler(self, num_features):
        stdsc = StandardScaler()
        for feature in num_features:
            y = stdsc.fit_transform(self.df[[feature]].values)
            self.df[feature] = y
        
    
fe = FE(all_df)

In [ ]:
# 列を追加する系の処理
# fe.fit_target()
fe.separete_cabin()

In [ ]:
features = [col for col in fe.df.columns]
features.remove(CFG.target)
features.remove('PassengerId')
features.remove('Name')

In [ ]:
cat_features = []
num_features = []
for feature in features:
    if all_df[feature].dtype == float:
        num_features.append(feature)
    else:
        cat_features.append(feature)


## Scaling and encoding

In [ ]:
all_features = cat_features + num_features
fe.fill_all_na(all_features)
cat_features = fe.label_encoding(cat_features)
all_features = cat_features + num_features
fe.standard_scaler(num_features)

In [ ]:
all_df = fe.df

In [ ]:
train_len = train.shape[0]
train = all_df[:train_len]
test = all_df[train_len:]

## Check Data

In [ ]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,...,Transported,Cabin1,Cabin2,Cabin3,HomePlanet_enc,Cabin_enc,Destination_enc,Cabin1_enc,Cabin2_enc,Cabin3_enc
0,0001_01,Europa,0,B/0/P,TRAPPIST-1e,0.730138,0,-0.340254,-0.281807,-0.292321,...,0,B,0,P,1,208,2,1,0,0
1,0002_01,Earth,0,F/0/S,TRAPPIST-1e,-0.279474,0,-0.170406,-0.276067,-0.249523,...,1,F,0,S,0,3241,2,5,0,1
2,0003_01,Europa,0,A/0/S,TRAPPIST-1e,2.008981,1,-0.273250,1.998829,-0.292321,...,0,A,0,S,1,1,2,0,0,1
3,0003_02,Europa,0,A/0/S,TRAPPIST-1e,0.326293,0,-0.340254,0.536441,0.342803,...,0,A,0,S,1,1,2,0,0,1
4,0004_01,Earth,0,F/1/S,TRAPPIST-1e,-0.817934,0,0.131893,-0.237164,-0.033821,...,1,F,1,S,0,3243,2,5,1,1


In [ ]:
train[CFG.target].head()

0    0
1    1
2    0
3    0
4    1
Name: Transported, dtype: int64

In [ ]:
display(train[train[all_features].isna().any(axis=1)])
display(test[test[all_features].isna().any(axis=1)])

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,...,Transported,Cabin1,Cabin2,Cabin3,HomePlanet_enc,Cabin_enc,Destination_enc,Cabin1_enc,Cabin2_enc,Cabin3_enc


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,...,Transported,Cabin1,Cabin2,Cabin3,HomePlanet_enc,Cabin_enc,Destination_enc,Cabin1_enc,Cabin2_enc,Cabin3_enc


In [ ]:
train.dtypes

PassengerId         object
HomePlanet          object
CryoSleep            int64
Cabin               object
Destination         object
Age                float64
VIP                  int64
RoomService        float64
FoodCourt          float64
ShoppingMall       float64
Spa                float64
VRDeck             float64
Name                object
Transported          int64
Cabin1              object
Cabin2              object
Cabin3              object
HomePlanet_enc       int64
Cabin_enc            int64
Destination_enc      int64
Cabin1_enc           int64
Cabin2_enc           int64
Cabin3_enc           int64
dtype: object

## Network implementation

In [ ]:
class Model(pl.LightningModule):
    # 親クラスがtorch.nn.moduleではなくLightningModuleになってる。
    def __init__(self, in_size, learning_rate, num_targets=1, hidden_size=128):
        # 6層のニューラルネットワーク
        # 活性化関数はswish

        # 入力層：in_size
        # 隠れ層：hidden_size
        # 出力層：num_targets
        super().__init__()
        self.in_size = in_size
        self.lr = learning_rate
        self.num_targets = num_targets
        self.hidden_size = hidden_size

        self.fc1 = nn.Linear(self.in_size, self.hidden_size)
        self.fc2 = nn.Linear(self.hidden_size, self.hidden_size)
        self.fc3 = nn.Linear(self.hidden_size, self.hidden_size)
        self.fc4 = nn.Linear(self.hidden_size, self.hidden_size)
        self.fc5 = nn.Linear(self.hidden_size, 16)
        self.fc6 = nn.Linear(16, self.num_targets)
        self.relu = F.relu
        self.swish = F.hardswish
        self.sigmoid = F.sigmoid
        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(0.2)  # 過学習対策、たまに0にしちゃう
        self.train_acc = torchmetrics.AUC(reorder=True)
        self.valid_acc = torchmetrics.AUC(reorder=True)

    def forward(self, x):
        # 予測する際に使用する
        # 本来、model(data)で呼ばれる処理
        # lightningでは、forward(data)とvalidation_step等で呼ばれる
        x = self.flatten(x)  # 多次元配列を1次元化
        x = self.dropout(self.swish(self.fc1(x)))
        x = self.dropout(self.swish(self.fc2(x)))
        x = self.dropout(self.swish(self.fc3(x)))
        x = self.dropout(self.swish(self.fc4(x)))
        x = self.dropout(self.swish(self.fc5(x)))
        x = self.sigmoid(self.fc6(x))  # 最後は活性化関数なし
        return x

    def training_step(self, batch, batch_idx):
        # バッチ処理用
        # DataLoaderをiterationして出力したbatchを受取
        # criterionで計算したlossを出力する
        X, y = batch
        y_hat = self(X).squeeze(1)  # forwardで予測した結果を、squeezeで1次元化
        #loss = F.binary_cross_entropy_with_logits(y_hat, y)
        loss = F.binary_cross_entropy(y_hat, y)
        self.log('loss', loss)
        return {'loss': loss}

    def validation_step(self, batch, batch_idx):
        X, y = batch
        y_hat = self(X).squeeze(1)
        #val_loss = F.binary_cross_entropy_with_logits(y_hat, y)
        val_loss = F.binary_cross_entropy(y_hat, y)
        results = {'val_loss': val_loss}
        return results
    
    def validation_epoch_end(self, validation_step_outputs):
        # validationの、各epochが終了した後に呼ばれる処理
        avg_loss = torch.stack([x['val_loss'] for x in validation_step_outputs]).mean()
        self.log('val_loss', avg_loss)

    def predict_step(self, X, batch_idx: int, dataloader_idx: int = None):
        # ??
        # 損失関数で、シグモイドをかけているので、ここでもかけてみる
        return self(X[0])

    def configure_optimizers(self):
        # optimizerを定義する
        optimizer = AdamW(
            self.parameters(),
            lr=self.lr,
            eps=1e-8,
            weight_decay=1e-6,
            amsgrad=False
        )
        # よくわからない・・・
        # バッチごとに学習率をコントロールしているのかも
        lr_scheduler = {
            "scheduler": ReduceLROnPlateau(
                optimizer,
                mode='min',
                factor=CFG.factor,  # 学習率の逓減率
                patience=CFG.lr_patience,
                min_lr=CFG.min_lr  # 最小学習率
            ),
            "interval": "epoch",
            "frequency": 1,
            "monitor": "val_loss",
            "strict": True,
            "name": "Learning Rate",
        }
        return [optimizer], [lr_scheduler]

In [24]:
# クロスバリデーションを行う
skf = StratifiedKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed)

for fold, (trn_idx, val_idx) in enumerate(skf.split(train, train[CFG.target])):
    print('Fold:', fold)
    # foldに対応したpd.DataFrameを抽出する
    X_train, y_train = train[all_features].iloc[trn_idx], train[CFG.target].iloc[trn_idx]
    X_valid, y_valid = train[all_features].iloc[val_idx], train[CFG.target].iloc[val_idx]
    X_test = test[all_features]

    # TensorDataset: 入力データと教師データが入ったデータセット
    # DataFrame.values: pd.DataFrameをnp.ndarrayに変換
    # FloatTensor: np.ndarrayをテンソル型に変換
    train_ds = TensorDataset(torch.FloatTensor(X_train.values), torch.FloatTensor(y_train.values))
    valid_ds = TensorDataset(torch.FloatTensor(X_valid.values), torch.FloatTensor(y_valid.values))

    # DataLoader: データセットをbatchで取り出す
    # 遅いらしい・・・？
    # https://qiita.com/bauer/items/98cb096e9fe585e7a926
    train_dl = DataLoader(train_ds, batch_size=CFG.batch_size, shuffle=True, num_workers=CFG.workers)
    valid_dl = DataLoader(valid_ds, batch_size=CFG.batch_size, shuffle=False, num_workers=CFG.workers)

    # インスタンス化
    model = Model(in_size=X_train.shape[1],
                  learning_rate=CFG.learning_rate,
                  num_targets=1,
                  hidden_size=32,
                 )
    print(ModelSummary(model))

    # モデルを保存する何からしい
    # 何かがキックで、モデルが保存される
    # fold別に保存するので、今回は10個出来上がる
    checkpoint_callback = ModelCheckpoint(
        dirpath=CFG.model_path,
        filename=f'model_{fold}_' + '{val_auc:.6f}',
        monitor='val_loss',  # lossに対して処理する
        mode='min',  # lossの最小値が更新されたらモデルを保存する
        save_weights_only=True)

    # ログを出す設定っぽい
    logger = TensorBoardLogger(
        save_dir=os.getcwd(),
        version=fold,
        name=CFG.tb_log_name
    )

    # 学習を初期で止める何からしい
    early_stop_callback = EarlyStopping(
        monitor='loss',
        min_delta=0.00,
        patience=CFG.es_patience,
        verbose=False,
        mode='min'
    )

    # 学習を監視する何からしい
    lr_monitor = LearningRateMonitor(
        # logging_interval='step'
        logging_interval='epoch'  # 違いが判らん
    )

    # 学習を実行するやつらしい
    trainer = pl.Trainer(
        fast_dev_run=False,  # デバッグ用のやつ
        max_epochs=CFG.epochs,
        gpus=1,  # gpuを使う設定っぽい
        # tpu_cores=1,
        precision=32,  # 32bit浮動小数点
        callbacks=[checkpoint_callback, lr_monitor, early_stop_callback],
        logger=logger,
        enable_progress_bar=False
     )

    # 学習開始
    trainer.fit(model, train_dl, valid_dl)
    # バリデーションを実行
    # trainer.validate(model, valid_dl, verbose=True)
    trainer.validate(model, valid_dl)

    # 学習したモデルでvalidationデータを予測する
    # 予測した結果をdfに記録する。
    preds = trainer.predict(model, valid_dl)
    train.loc[val_idx, CFG.pred] = torch.cat(preds).cpu().numpy()
    
    # 学習したモデルでテストデータを予測する
    # 予測した結果をdfに記録する。
    test_ds = TensorDataset(torch.FloatTensor(X_test.values))
    test_dl = DataLoader(test_ds, batch_size=CFG.batch_size, shuffle=False, num_workers=CFG.workers)
    preds = trainer.predict(model, test_dl)
    test[CFG.test_pred[fold]] = torch.cat(preds).cpu().numpy().flatten()

    # TODOいい感じのログをだしたいよお
    
    del model, trainer, X_train, X_valid, y_train, y_valid, train_ds, valid_ds, train_dl, valid_dl
    gc.collect()
    torch.cuda.empty_cache()

auc = roc_auc_score(train[CFG.target], train[CFG.pred])
print(f"auc: {auc:.6f}")

Fold: 0
  | Name      | Type    | Params
--------------------------------------
0 | fc1       | Linear  | 480   
1 | fc2       | Linear  | 1.1 K 
2 | fc3       | Linear  | 1.1 K 
3 | fc4       | Linear  | 1.1 K 
4 | fc5       | Linear  | 528   
5 | fc6       | Linear  | 17    
6 | flatten   | Flatten | 0     
7 | dropout   | Dropout | 0     
8 | train_acc | AUC     | 0     
9 | valid_acc | AUC     | 0     
--------------------------------------
4.2 K     Trainable params
0         Non-trainable params
4.2 K     Total params
0.017     Total estimated model params size (MB)


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         val_loss          │     0.486453115940094     │
└───────────────────────────┴───────────────────────────┘

Fold: 1
  | Name      | Type    | Params
--------------------------------------
0 | fc1       | Linear  | 480   
1 | fc2       | Linear  | 1.1 K 
2 | fc3       | Linear  | 1.1 K 
3 | fc4       | Linear  | 1.1 K 
4 | fc5       | Linear  | 528   
5 | fc6       | Linear  | 17    
6 | flatten   | Flatten | 0     
7 | dropout   | Dropout | 0     
8 | train_acc | AUC     | 0     
9 | valid_acc | AUC     | 0     
--------------------------------------
4.2 K     Trainable params
0         Non-trainable params
4.2 K     Total params
0.017     Total estimated model params size (MB)


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         val_loss          │    0.5003978610038757     │
└───────────────────────────┴───────────────────────────┘

Fold: 2
  | Name      | Type    | Params
--------------------------------------
0 | fc1       | Linear  | 480   
1 | fc2       | Linear  | 1.1 K 
2 | fc3       | Linear  | 1.1 K 
3 | fc4       | Linear  | 1.1 K 
4 | fc5       | Linear  | 528   
5 | fc6       | Linear  | 17    
6 | flatten   | Flatten | 0     
7 | dropout   | Dropout | 0     
8 | train_acc | AUC     | 0     
9 | valid_acc | AUC     | 0     
--------------------------------------
4.2 K     Trainable params
0         Non-trainable params
4.2 K     Total params
0.017     Total estimated model params size (MB)


...

Fold: 8
  | Name      | Type    | Params
--------------------------------------
0 | fc1       | Linear  | 480   
1 | fc2       | Linear  | 1.1 K 
2 | fc3       | Linear  | 1.1 K 
3 | fc4       | Linear  | 1.1 K 
4 | fc5       | Linear  | 528   
5 | fc6       | Linear  | 17    
6 | flatten   | Flatten | 0     
7 | dropout   | Dropout | 0     
8 | train_acc | AUC     | 0     
9 | valid_acc | AUC     | 0     
--------------------------------------
4.2 K     Trainable params
0         Non-trainable params
4.2 K     Total params
0.017     Total estimated model params size (MB)


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         val_loss          │    0.49457791447639465    │
└───────────────────────────┴───────────────────────────┘

Fold: 9
  | Name      | Type    | Params
--------------------------------------
0 | fc1       | Linear  | 480   
1 | fc2       | Linear  | 1.1 K 
2 | fc3       | Linear  | 1.1 K 
3 | fc4       | Linear  | 1.1 K 
4 | fc5       | Linear  | 528   
5 | fc6       | Linear  | 17    
6 | flatten   | Flatten | 0     
7 | dropout   | Dropout | 0     
8 | train_acc | AUC     | 0     
9 | valid_acc | AUC     | 0     
--------------------------------------
4.2 K     Trainable params
0         Non-trainable params
4.2 K     Total params
0.017     Total estimated model params size (MB)


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         val_loss          │    0.5970510840415955     │
└───────────────────────────┴───────────────────────────┘

auc: 0.844082


In [25]:
# この時点でtestには各CVで計算した予測結果が編集されてはいる
# ただ、lossが最も小さかったモデルを引っ張ってきてそれで学習しなおす?(ほんと?)
test.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,...,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7,pred_8,pred_9
8693,0013_01,Earth,1,G/3/S,TRAPPIST-1e,-0.077552,0,-0.340254,-0.281807,-0.292321,...,0.638247,0.526028,0.773565,0.722838,0.630299,0.749715,0.766508,0.673848,0.682852,0.456084
8694,0018_01,Earth,0,F/4/S,TRAPPIST-1e,-0.616012,0,-0.340254,-0.276067,-0.292321,...,0.118457,0.180229,0.140067,0.146164,0.152830,0.122101,0.172360,0.159312,0.146547,0.444566
8695,0019_01,Europa,1,C/0/S,55 Cancri e,0.191678,0,-0.340254,-0.281807,-0.292321,...,0.723210,0.328592,0.966091,0.910793,0.919544,0.989623,0.993480,0.890232,0.985589,0.804150
8696,0021_01,Europa,0,C/1/S,TRAPPIST-1e,0.662831,0,-0.340254,3.960584,-0.292321,...,0.717489,0.593589,0.966045,0.918936,0.923676,0.986067,0.993263,0.953186,0.981407,0.808901
8697,0023_01,Earth,0,F/5/S,TRAPPIST-1e,-0.548704,0,-0.324672,-0.281807,0.794751,...,0.581183,0.518789,0.566993,0.501626,0.730080,0.605311,0.538841,0.653661,0.683688,0.440691


In [26]:
models = np.sort(glob.glob(f"./{CFG.model_path}/*.ckpt"))
trainer = pl.Trainer(gpus=1)

# fold別に作った10個のモデルをロードする
for model_name in models:
    X_test = test[all_features]
    model = Model(in_size=X_test.shape[1],
              learning_rate=CFG.learning_rate,
              num_targets=1,
              hidden_size=32,
             )
    model.load_state_dict(torch.load(model_name)['state_dict'])
    test_ds = TensorDataset(torch.FloatTensor(X_test.values))
    test_dl = DataLoader(test_ds, batch_size=CFG.batch_size, shuffle=False, num_workers=CFG.workers)

    preds = trainer.predict(model, test_dl)
    test[CFG.test_pred[fold]] = torch.cat(preds).cpu().numpy().flatten()



Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

In [27]:
test[CFG.test_pred].mean(axis=1).nunique()

4265

In [28]:
import scipy
scipy.stats.mode(test[CFG.test_pred].mean(axis=1).values)[0].item()

0.2261662781238556

In [29]:
sub[CFG.target] = test[CFG.test_pred].mean(axis=1).values

In [30]:
sub[CFG.target]

0       0.662339
1       0.158079
2       0.854320
3       0.886011
4       0.587252
          ...   
4272    0.643572
4273    0.441612
4274    0.949261
4275    0.640002
4276    0.651967
Name: Transported, Length: 4277, dtype: float32

In [31]:
sub[CFG.target] = test[CFG.test_pred].mean(axis=1).values
h = sub[CFG.target].median()
sub[CFG.target] = (sub[CFG.target] > h)
sub.to_csv("submission.csv", index=False)
sub

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True


In [32]:
sub[CFG.target].sum()

2138